In [102]:
import math
import random
import bisect
import os
from datetime import datetime
import csv
import sys

In [123]:
def matrix_full(Matrix):

    pop = [Matrix[i][j].pop for i in range(rows) for j in range(cols)]
    
    if all(pop):
        return 1
    else:
        return 0

def print_progress(iteration, total, prefix='', suffix='', decimals=1, bar_length=100):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        bar_length  - Optional  : character length of bar (Int)
    """
    str_format = "{0:." + str(decimals) + "f}"
    percents = str_format.format(100 * (iteration / float(total)))
    filled_length = int(round(bar_length * iteration / float(total)))
    bar = '#' * filled_length + '-' * (bar_length - filled_length)

    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percents, '%', suffix)),

    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()

def create_matrix(cols, rows):
    Matrix = [[0] * cols for i in range(rows)]
    for i in range(rows):
        for j in range(cols):
            Matrix[i][j] = Cell(i, j, pop = False)
    return Matrix

def cdf(weights):
    total = sum(weights)
    result = []
    cumsum = 0
    for w in weights:
        cumsum += w
        result.append(cumsum / total)
    return result

def choice(population, weights):
    assert len(population) == len(weights)
    cdf_vals = cdf(weights)
    x = random.random()
    idx = bisect.bisect(cdf_vals, x)
    return population[idx]
        
def cline(s, results, rows, cols, steps, pA, pB, Matrix, max_p_create, bot, min_K, max_K, r):
        
    for step in range(steps):
        
        pop_list = [(i, j) for j in range(cols) for i in range(rows) if Matrix[i][j].pop]
        mat_full = matrix_full(Matrix)
        
        for pop in pop_list:
           
            i, j = pop[0], pop[1]
            
#             if step == 0:
                
#                 K = max_K
#                 pass
                
#             else:
                
            size = Matrix[i][j].population.size
            pA = Matrix[i][j].population.allele_freq(Matrix[i][j].population.locus_A)
            pB = Matrix[i][j].population.allele_freq(Matrix[i][j].population.locus_B)
            phen = Matrix[i][j].population.phenotype(pA, pB)
            K = Matrix[i][j].Distance_calc(0, 0, rows, cols, max_mig_rate, min_K, max_K)[1]

            results.append([s, i, j, step, round(pA, 3), round(pB, 3), 
                        round(phen, 3), max_p_create, K, round(r, 3), 
                        bot, max_mig_rate, mat_full, size, min_K, max_K])
        
#             K = Matrix[i][j].Distance_calc(0, 0, rows, cols, max_mig_rate, min_K, max_K)[1]
#                 print Matrix[i][j].population.size
            Matrix[i][j].population.size = Matrix[i][j].population.pop_growth(Matrix[i][j].population.size, K, r)

            pA1 = Matrix[i][j].alleles_next_gen(rows, cols, pop_list, max_mig_rate, min_K, max_K)[0]
            pB1 = Matrix[i][j].alleles_next_gen(rows, cols, pop_list, max_mig_rate, min_K, max_K)[1]

            Matrix[i][j].population.locus_A = Matrix[i][j].population.sample_alleles(pA1, r, 'Aa')
            Matrix[i][j].population.locus_B = Matrix[i][j].population.sample_alleles(pB1, r, 'Bb')
            
            Matrix[i][j].create_population(rows, cols, Matrix, max_p_create, bot, K, r)
           
        
#             size = Matrix[i][j].population.size
#             pA = Matrix[i][j].population.allele_freq(Matrix[i][j].population.locus_A)
#             pB = Matrix[i][j].population.allele_freq(Matrix[i][j].population.locus_B)
#             phen = Matrix[i][j].population.phenotype(pA, pB)

#             results.append([s, i, j, step, round(pA, 3), round(pB, 3), 
#                             round(phen, 3), max_p_create, K, round(r, 3), 
#                             bot, max_mig_rate, mat_full, size, min_K, max_K])
                           

def simulate():
    
    print os.getpid()
    
    os.chdir(export_path)
    datestring = datetime.strftime(datetime.now(), '%Y%m%d')
    
    with open(datestring + "_SEC_Drift.Migration_(pA%.2f)(pB%.2f).csv" % (pA, pB), "wb") as f:
        writer = csv.writer(f)
        writer.writerow(["Sim","x","y","Generation","pA","pB","Cyan","p_create","K", "r", "bot", "Mig_rate", "Mat_full", "Pop_size", "min_K", "max_K"])
    
        print_progress(0, sims, prefix='', suffix='', decimals=1, bar_length=100)
        for s in range(sims):
                        
            results = []

#             locus_A = ( [ 'A' ] * int( N * pA ) ) + ( [ 'a' ] * int( round( N * qA ) ) )
#             locus_B = ( [ 'B' ] * int( N * pB ) ) + ( [ 'b' ] * int( round( N * qB ) ) ) 

            Matrix = create_matrix(cols, rows)
            for i in range(rows):
                for j in range(cols):
                    Matrix[i][j].pop = True
                    K = Matrix[i][j].Distance_calc(0, 0, rows, cols, max_mig_rate, min_K, max_K)[1]
                    locus_A = ( [ 'A' ] * int( K * pA ) ) + ( [ 'a' ] * int( round( K * qA ) ) )  # Re-initialize initial allele lists.
                    locus_B = ( [ 'B' ] * int( K * pB ) ) + ( [ 'b' ] * int( round( K * qB ) ) ) 
                    Matrix[i][j].population = Population(K, locus_A, locus_B)
            
#             Matrix = create_matrix(cols, rows)
#             Matrix[0][0].pop = True
#             Matrix[0][0].population = Population(N, locus_A, locus_B)
            
            cline(s, results, rows, cols, steps, pA, pB, Matrix, max_p_create, bot, min_K, max_K, r) 
                    
            write_to_csv(writer, results)
        
            del results

            print_progress(s + 1, sims, prefix='', suffix='', decimals=1, bar_length=100)
            
#                     print K, Matrix[i][j].pop, Matrix[i][j].population.size, Matrix[i][j].population.locus_A

def write_to_csv(writer, results):

    for z in results:

        writer.writerow([z[0], z[1], z[2], z[3], z[4], z[5], z[6], z[7], z[8], z[9], z[10], z[11], z[12], z[13], z[14], z[15]])

In [124]:
class Cell(object):    
    def __init__( self, i, j, pop ):
        self.i = i
        self.j = j

        self.pop = False
        self.population = Population( 0, [], [] )
    
    def prob_create(self, K, max_p_create):
        m = (max_p_create - 0)/(K - 0) # Slope. Assumes close to no migration at max distance. Relized migration at max distance may be slightly greater than 0 due to rounding.
        Size = self.population.size
        p_create = Size * m
        return float(p_create)

    
    def create_population( self, rows, cols, Matrix, max_p_create, bot, K, r ):
        population = '10'
        number_of_items_to_pick = 1
        p_create = self.prob_create( K, max_p_create )
        weights = [ p_create, ( 1 - p_create ) ]
        create = [ choice( population, weights ) for i in range( number_of_items_to_pick ) ]
        if create[0] == '1':
            neighbors = [ ( x2, y2 ) for x2 in range( self.i - 1, self.i + 2 )
                    for y2 in range( self.j - 1, self.j + 2 )
                    if ( -1 < self.i <= rows - 1 and
                    -1 < self.j <= cols - 1 and
                    ( self.i != x2 or self.j != y2 ) and
                    ( 0 <= x2 <= rows - 1 ) and
                    ( 0 <= y2 <= cols - 1 ) ) ]

            Nlist_red = []
            for item in neighbors:
                i, j = item[ 0 ], item[ 1 ]
                if not Matrix[ i ][ j ].pop:
                    Nlist_red.append( item )

            if not Nlist_red:
                pass
            else:
            # Otherwise, select and empty neighboring cell at random and place new population
                Nsam = random.randint( 0, len( Nlist_red ) - 1 )
                i, j = Nlist_red[ Nsam ][ 0 ], Nlist_red[ Nsam ][ 1 ]
                Matrix[ i ][ j ].pop = True
                new_size = int( math.ceil( bot * self.population.size ) )
                new_locus_A = self.population.sample_population( self.population.locus_A, new_size )
                new_locus_B = self.population.sample_population( self.population.locus_B, new_size )
                Matrix[ i ][ j ].population = Population( new_size, new_locus_A, new_locus_B )
        else:
            pass
         
    def Distance_calc(self, y, x, rows, cols, max_mig_rate, min_K, max_K):
                
        dist = (((y - self.i) ** 2) + ((x - self.j) ** 2)) ** (0.5)
        
        max_i = rows - 1
        max_j = cols - 1
        
        start_x = 0
        start_y = 0
        
        max_dis = (((max_j - start_x) ** 2) + ((max_i - start_y) ** 2)) ** (0.5) # Max distance between populations

        m_mig = (max_mig_rate - 0)/(max_dis - 0) # Slope. Assumes close to no migration at max distance. Relized migration at max distance may be slightly greater than 0 due to rounding.
        
        mig_prop = max_mig_rate - m_mig * dist
        
        m_k = (min_K - max_K) / (max_dis - 0)
        
        K = max_K + m_k * dist

        return round(mig_prop, 4), int( math.ceil ( K ) )
                                                      
    
    def migration_weighted(self, rows, cols, pop_list, max_mig_rate, min_K, max_K):
        
        migration_weighted = []
        allele_weighted_A = []
        allele_weighted_B = []
        pop_size = []

        for pop in pop_list:
            y, x = pop[0], pop[1]
            if y == self.i and x == self.j:
#                 print y, self.i, x, self.j, "SAME"
                pass
            else:
#                 print y, self.i, x, self.j, "DIFFERENT"
#                 print (self.i, self.j), (y, x), self.Distance_calc(y, x, rows, cols)[0]
                migration_weighted.append(self.Distance_calc(y, x, rows, cols, max_mig_rate, min_K, max_K)[0])
                allele_weighted_A.append(self.population.allele_freq(self.population.locus_A)) # Append alleles 'A' frequency to list
                allele_weighted_B.append(self.population.allele_freq(self.population.locus_B)) # Append allele 'B' frequency to list
                pop_size.append(self.population.size)
        migration_weighted = sum(migration_weighted[g] * pop_size[g] / sum(pop_size) for g in range(len(migration_weighted))) # Weighted migration rate
        allele_weighted_A = sum(allele_weighted_A[g] * pop_size[g] / sum(pop_size) for g in range(len(allele_weighted_A))) # Weighted allele 'A'
        allele_weighted_B = sum(allele_weighted_B[g] * pop_size[g] / sum(pop_size) for g in range(len(allele_weighted_B))) # Weighted allele 'B'
        
        pA1 = ((1 - migration_weighted) * self.population.allele_freq(self.population.locus_A)) + (migration_weighted * allele_weighted_A) # Probability of sampling 'A' in next generation
        pB1 = ((1 - migration_weighted) * self.population.allele_freq(self.population.locus_B)) + (migration_weighted * allele_weighted_B) # Probability of sampling 'B' in next generation

        # Ensure that negative probabilities do not occur.
        if pA1 < 0:
            pA1 = 0
        elif pA1 > 1.0:
            pA1 = 1.0
        if pB1 < 0:
            pB1 = 0
        elif pB1 > 1.0:
            pB1 = 1.0

        return pA1, pB1
    
    def alleles_next_gen(self, rows, cols, pop_list, max_mig_rate, min_K, max_K):

        pA1_mig = self.migration_weighted(rows, cols, pop_list, max_mig_rate, min_K, max_K)[0]
        pB1_mig = self.migration_weighted(rows, cols, pop_list, max_mig_rate, min_K, max_K)[1]
        
        return pA1_mig, pB1_mig


In [125]:
class Population( object ):
    
    def __init__( self, size, locus_A, locus_B ):
        self.size = size
        self.locus_A = locus_A
        self.locus_B = locus_B
        
    def pop_growth( self, size, K, r ):
        K = float( K )
        self.size = self.size * K / ( self.size + ( K - self.size ) * math.exp( -r ) )
        self.size = int( math.ceil( self.size ) )
        return self.size
    
    def phenotype( self, locus_A, locus_B ):
        qA = 1 - self.allele_freq( self.locus_A )
        qB = 1 - self.allele_freq( self.locus_B )
        mut= qA ** 2 + qB ** 2 - ( qA ** 2 * qB ** 2 )
        WT = 1 - mut
        return WT
        
    @staticmethod
    def allele_freq( locus ):
        p = sum( 1 * i.isupper() for i in locus ) / float( len( locus ) )
        return p 
    
    @staticmethod
    def sample_population( locus, N ):
        new_locus = [ random.choice( locus ) for _ in range( N )]
        return new_locus
    
    def sample_alleles(self, allele, r, possibilities):
        number_of_items_to_pick = self.size # Number to sample. Corresponds to next generation's size.
        weights = [allele, (1 - allele)] # Sampling probabilities. Returned by 'alleles_next_gen'
        return [choice(possibilities, weights) for i in range(number_of_items_to_pick)]

In [126]:
rows = 1
cols = 20
max_K = 1000
min_K = 10
r = float( math.log( 2 ) )
N = 100
pA, pB = 0.5, 0.5
qA, qB = 1 - pA, 1 - pB
# locus_A = ( [ 'A' ] * int( K * pA ) ) + ( [ 'a' ] * int( round( K * qA ) ) )  # Re-initialize initial allele lists.
# locus_B = ( [ 'B' ] * int( K * pB ) ) + ( [ 'b' ] * int( round( K * qB ) ) ) 
max_p_create = 0.0
max_mig_rate = 0.0
bot = 1.0
steps = 40
export_path = "/Users/jamessantangelo/Desktop/CSV"
sims = 1

In [127]:
simulate()

13911
 |####################################################################################################| 100.0% 


In [48]:
Matrix = create_matrix(cols, rows)
# locus_A = ( [ 'A' ] * int( K * pA ) ) + ( [ 'a' ] * int( round( K * qA ) ) )  # Re-initialize initial allele lists.
# locus_B = ( [ 'B' ] * int( K * pB ) ) + ( [ 'b' ] * int( round( K * qB ) ) ) 
for i in range(rows):
    for j in range(cols):
        Matrix[i][j].pop = True
        K = Matrix[i][j].Distance_calc(0, 0, rows, cols, max_mig_rate, min_K, max_K)[1]
        locus_A = ( [ 'A' ] * int( K * pA ) ) + ( [ 'a' ] * int( round( K * qA ) ) )  # Re-initialize initial allele lists.
        locus_B = ( [ 'B' ] * int( K * pB ) ) + ( [ 'b' ] * int( round( K * qB ) ) ) 
        Matrix[i][j].population = Population(K, locus_A, locus_B)
        
        
        print K, Matrix[i][j].pop, Matrix[i][j].population.size, Matrix[i][j].population.locus_A


1000 True 1000 ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',

In [101]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [65]:
y, x = 0, 0
rows, cols = 1, 20
max_mig_rate = 0
min_K = 10
max_K = 1000
Matrix = create_matrix(cols, rows)

for i in range(rows):
    for j in range(cols):
#         Matrix[i][j].pop = True

        print Matrix[i][j].Distance_calc(y, x, rows, cols, max_mig_rate, min_K, max_K)

0 19 19.0
(0.0, 1000)
0 19 19.0
(0.0, 948)
0 19 19.0
(0.0, 896)
0 19 19.0
(0.0, 844)
0 19 19.0
(0.0, 792)
0 19 19.0
(0.0, 740)
0 19 19.0
(0.0, 688)
0 19 19.0
(0.0, 636)
0 19 19.0
(0.0, 584)
0 19 19.0
(0.0, 532)
0 19 19.0
(0.0, 479)
0 19 19.0
(0.0, 427)
0 19 19.0
(0.0, 375)
0 19 19.0
(0.0, 323)
0 19 19.0
(0.0, 271)
0 19 19.0
(0.0, 219)
0 19 19.0
(0.0, 167)
0 19 19.0
(0.0, 115)
0 19 19.0
(0.0, 63)
0 19 19.0
(0.0, 10)
